## Word2Vec Tutorial
参考原blog: https://rare-technologies.com/word2vec-tutorial/  
这是一个简单的教程，如何使用 gensim 中的 Word2Vec 来训练自己的词向量。

### 源代码
先大致看一下源代码，其中设计到很多参数，接下来会一一了解。
```python
class Word2Vec(BaseWordEmbeddingsModel):
    """Class for training, using and evaluating neural networks described in https://code.google.com/p/word2vec/

    If you're finished training a model (=no more updates, only querying)
    then switch to the :mod:`gensim.models.KeyedVectors` instance in wv

    The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save()` and
    :meth:`~gensim.models.word2vec.Word2Vec.load()` methods, or stored/loaded in a format
    compatible with the original word2vec implementation via `wv.save_word2vec_format()`
    and `Word2VecKeyedVectors.load_word2vec_format()`.

    """

    def __init__(self, sentences=None, size=100, alpha=0.025, window=5, min_count=5,
                 max_vocab_size=None, sample=1e-3, seed=1, workers=3, min_alpha=0.0001,
                 sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=hash, iter=5, null_word=0,
                 trim_rule=None, sorted_vocab=1, batch_words=MAX_WORDS_IN_BATCH, compute_loss=False, callbacks=()):
```

### 输入数据
word2vec 的输入数据，也就是第一个参数 sentences，其形式是一系列 sentence， 每个sentence 是元素为 words(utf-8 strings) 的 list

In [2]:
# import modules & set up logging

import gensim, logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level=logging.INFO)

sentences = [['first', 'sentence'],['second', 'sentence']]

# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

Gensim只需要在迭代时输入必须按顺序提供句子。 无需将所有内容保存在RAM中：我们可以提供一个句子，处理它，忘记它，加载另一个句子......

例如，如果我们的输入是在磁盘上的几个文件中，每行一个句子，那么我们可以逐行处理输入文件，而不是将所有内容加载到内存中：

In [1]:
import os
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
        
    def __iter__(self):
#         for fname in os.listdir(self.dirname):
#             for line in open(os.path.join(self.dirname, fname)):
#                 yield line.split()
        # 这里只有一个 文件
        for line in open(self.dirname):
                yield line.strip().split()
                
sentences = MySentences("train/train.en") # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences)

假设我们想要进一步预处理文件中的单词 - 转换为unicode，小写，删除数字，提取命名实体......所有这些都可以在MySentences迭代器中完成，而word2vec不需要知道。 所需要的只是输入产生一个句子（utf8单词列表）。

调用 Word2Vec（sentences，iter = 1）将在句子迭代器上运行两遍（或者，通常iter + 1遍;默认iter = 5）。 第一遍收集单词及其频率以构建内部字典树结构。 第二次和随后的遍历训练神经模型。 这两次（或者，iter + 1）遍历也可以手动初始化，以防输入流不可重复（只能承担一次遍历），并且可以通过其他方式初始化词汇表：

In [ ]:
model = gensim.models.Word2Vec(iter=1) # # an empty model, no training yet
model.build_vocab(some_sentences)  # can be a non-repeatable, 1-pass generator
model.train(other_sentences)  # can be a non-repeatable, 1-pass generator

如果对 python 中迭代器 iterators, 迭代序列 iterables 以及生成器 generator 有疑惑，可以参考这篇 [ Data Streaming in Python.](https://rare-technologies.com/data-streaming-in-python-generators-iterators-iterables/)

### 训练

Word2Vec 的参数对训练速度和训练质量有会有一定影响。

#### parameters: min_count
```python
min_count : int, Ignores all words with total frequency lower than this.
```
其中之一是修剪内部字典(pruning the internal dictionary)。 在十亿字的语料库中只出现一次或两次的单词可能是没有意义的错别字和垃圾字。此外，没有足够的数据对这些单词进行任何有意义的训练（word2vec的原理就是共现频率），因此最好忽略它们：

In [ ]:
model = Word2Vec(sentences, min_count=10)  # default value is 5

对于 min_count 一个合理的值是在 0～100 之间的，这取决与数据集的大小。

#### parameters: size
```python
size : int, Dimensionality of the feature vectors.
```
另一个参数是 size，也就是词向量的维度。在原博客对 size 的描述是 is the size of the NN layers, which correspond to the “degrees” of freedom the training algorithm has. 之所以称为 NN layers 是因为在google word2vec 模型中词向量的维度也就是隐藏层 h 的维度。

model = Word2Vec(sentences, size=200)  # default value is 100

size 越大需要的训练数据越多，但能得到更准确的模型。合理的大小是 10～100.

#### parameters: works 
训练并行化，以提高训练速度。

In [ ]:
model = Word2Vec(sentences, workers=4) # default = 1 worker = no parallelization

只有安装了Cython才有效的workers参数。 没有Cython，你只能使用一个核心因为GIL（并且word2vec训练将非常缓慢）。

#### 其他的参数
- sentences (iterable of iterables) – The sentences iterable can be simply a list of lists of tokens, but for larger corpora, consider an iterable that streams the sentences directly from disk/network. See BrownCorpus, Text8Corpus or LineSentence in word2vec module for such examples. If you don’t supply sentences, the model is left uninitialized – use if you plan to initialize it in some other way.
- sg (int {1, 0}) – Defines the training algorithm. If 1, skip-gram is employed; otherwise, CBOW is used.
- size (int) – Dimensionality of the feature vectors.
- window (int) – The maximum distance between the current and predicted word within a sentence.
- alpha (float) – The initial learning rate.
- min_alpha (float) – Learning rate will linearly drop to min_alpha as training progresses.
- seed (int) – Seed for the random number generator. Initial vectors for each word are seeded with a hash of the concatenation of word + str(seed). Note that for a fully deterministically-reproducible run, you must also limit the model to a single worker thread (workers=1), to eliminate ordering jitter from OS thread scheduling. (In Python 3, reproducibility between interpreter launches also requires use of the PYTHONHASHSEED environment variable to control hash randomization).
- min_count (int) – Ignores all words with total frequency lower than this.
- max_vocab_size (int) – Limits the RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM. Set to None for no limit.
- sample (float) – The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
- workers (int) – Use these many worker threads to train the model (=faster training with multicore machines).
- hs (int {1,0}) – If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
- negative (int) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
- cbow_mean (int {1,0}) – If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
- hashfxn (function) – Hash function to use to randomly initialize weights, for increased training reproducibility.
- iter (int) – Number of iterations (epochs) over the corpus.
- trim_rule (function) – Vocabulary trimming rule, specifies whether certain words should remain in the vocabulary, be trimmed away, or handled using the default (discard if word count < min_count). Can be None (min_count will be used, look to keep_vocab_item()), or a callable that accepts parameters (word, count, min_count) and returns either gensim.utils.RULE_DISCARD, gensim.utils.RULE_KEEP or gensim.utils.RULE_DEFAULT. Note: The rule, if given, is only used to prune vocabulary during build_vocab() and is not stored as part of the model.
- sorted_vocab (int {1,0}) – If 1, sort the vocabulary by descending frequency before assigning word indexes.
- batch_words (int) – Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
- compute_loss (bool) – If True, computes and stores loss value which can be retrieved using model.get_latest_training_loss().
- callbacks – List of callbacks that need to be executed/run at specific stages during training.

### 内存

At its core, word2vec model parameters are stored as matrices (NumPy arrays). Each array is #vocabulary (controlled by min_count parameter) times #size (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer size=200, the model will require approx. 100,000*200*4*3 bytes = ~229MB.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

#### 模型评估

Word2vec 的训练是无监督任务，因此并没有很好的方式来客观的判断结果的好坏，因此模型验证依赖于最终的应用。

Google 发布一个测试集，关于 20000 个句法和语义的测试样本，"A is to B as C is to D" 任务：
https://raw.githubusercontent.com/RaRe-Technologies/gensim/develop/gensim/test/test_data/questions-words.txt

gensim 也支持这样的验证集：

```python
    @deprecated("Method will be removed in 4.0.0, use self.wv.accuracy() instead")
    def accuracy(self, questions, restrict_vocab=30000, most_similar=None, case_insensitive=True):
```

In [ ]:
model.accuracy('/tmp/questions-words.txt')

参数 restrict_vocab 用来限制测试样本的数量。

最后，在这个测试集的表现好并不意味这 word2vec 在你自己的应用上表现就很好，反之亦然。所以最好的验证是直接在你自己的任务上。

### 存储和加载模型

In [ ]:
model.save('/tmp/mymodel')
new_model = gensim.models.Word2Vec.load('/tmp/mymodel')

在内部使用pickle，可选择将模型的内部大型NumPy矩阵直接从磁盘文件映射到虚拟内存中，以进行进程间内存共享。

此外，您可以使用其文本和二进制格式加载由原始C工具创建的模型：

In [ ]:
model = Word2Vec.load_word2vec_format('/tmp/vectors.txt', binary=False)
# using gzipped/bz2 input works too, no need to unzip:
model = Word2Vec.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)

#### 在线训练/恢复训练 online traing/resuming training
可以通过加载模型，并继续训练它在更多的句子上。

```python
    def train(self, sentences, total_examples=None, total_words=None,
              epochs=None, start_alpha=None, end_alpha=None, word_count=0,
              queue_factor=2, report_delay=1.0, compute_loss=False, callbacks=()):
        """Update the model's neural weights from a sequence of sentences (can be a once-only generator stream).
        For Word2Vec, each sentence must be a list of unicode strings. (Subclasses may accept other examples.)
```

In [ ]:
model = gensim.models.Word2Vec.load('/tmp/mymodel')
model.train(more_sentences)

In [ ]:
这里可能需要调整 train() 的参数 total_words

### 模型的使用

word2vec 支持查找词的最相似词,以及两个词的相似性。

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
[('queen', 0.50882536)]
model.doesnt_match("breakfast cereal dinner lunch";.split())
'cereal'
model.similarity('woman', 'man')
0.73723527

关于词向量，可以逐词访问。

In [ ]:
model['computer']  # raw NumPy vector of a word
array([-0.00449447, -0.00310097,  0.02421786, ...], dtype=float32)